In [1]:
import os
import sys
import numpy as np
import cv2

from utils.homography import compute_h, cor_p
from utils.triangulation import triangulation
from pathlib import Path
from shapely.geometry import Polygon

In [2]:
num_cams = 4
cam_poses = {} # key: cami, value: pose
for i in range(num_cams):
    with open(f'./camera_poses/{i:05d}.txt', 'r') as f:
        lines = f.readlines()
        pose = []
        for line in lines:
            data = list(map(float, line.split(" ")))
            pose.append(data)
        pose = np.array(pose)
        cam_poses[f'cam{i}'] = pose.reshape(4, 4)

In [3]:
yolo_outputs = {} # key: cami, value: yolo_output

for i in range(num_cams):

    # outputs of one camera:
    yolo_output = {} # key: class, value: yolo_output
    yolo_output['chair'], yolo_output['table'], yolo_output['person'] = [], [], []

    with open(f'./runs/detect/layout/cam{i}/labels/00000.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            data = list(map(float, line.split(" ")))

            # check object class
            if data[0] == 56 or data[0] == 11: # chair or bench
                yolo_output['chair'].append(np.array(data[1:5]))
            elif data[0] == 60: # table
                yolo_output['table'].append(np.array(data[1:5]))
            elif data[0] == 0: # person
                yolo_output['person'].append(np.array(data[1:5]))
        yolo_outputs[f'cam{i}'] = yolo_output

In [8]:
h, w = 1456, 1928

poses = [cam_poses[f'cam{i}'][:3, :] for i in range(num_cams)]

K = np.array([
        [975.813843, 0, 960.973816],
        [0, 975.475220, 729.893921],
        [0, 0, 1]
    ])

for idx in range(len(cor_p)):
    X = [cor_p[i][idx][0] for i in range(num_cams)]
    Y = [cor_p[i][idx][1] for i in range(num_cams)]
    print(X, Y)
    triangulated = triangulation(poses, X, Y)
    print(triangulated)
    for i in range(num_cams):
        reversed = K @ poses[i] @ np.array(triangulated.tolist() + [1])
        reversed = [reversed[0]/reversed[-1], reversed[1]/reversed[-1]]
        print(f"Reversed{i}: ", reversed)

[609, 1223, 1613, 1673] [1025, 1082, 893, 698]
[-2.22324864 -0.83021165  3.27884171]
Reversed0:  [299.314306524057, 482.90093805638975]
Reversed1:  [220.56496972177214, 216.87949992647611]
Reversed2:  [403.2184400857285, 210.67001604920713]
Reversed3:  [774.5617292071622, 326.952170277115]
[399, 568, 834, 1203] [632, 863, 897, 888]
[-1.65689084  0.2555697   3.50342011]
Reversed0:  [499.47706079732404, 801.0535038068356]
Reversed1:  [373.2010186967893, 555.5557830543232]
Reversed2:  [445.63817821232294, 484.35668206919325]
Reversed3:  [712.0086481127998, 524.6002201086619]
[1491, 1523, 1463, 1297] [819, 798, 669, 510]
[-0.81926374 -0.89068167  2.25822891]
Reversed0:  [606.9579336184303, 345.15086408756395]
Reversed1:  [155.41077004241947, 54.08820363821946]
Reversed2:  [151.86582391189, 88.81564975553185]
Reversed3:  [551.7675729739271, 210.33092961376343]
[683, 718, 775, 834] [549, 739, 755, 734]
[-0.24672675  0.06841967  3.22583904]
Reversed0:  [886.3391629182964, 750.5836365173292]
R